In [ ]:
# 設定影片資料夾
input_folder = "video"
output_folder = "output"

In [ ]:
####影片轉檔
import os
os.chdir('input_folder')  # 使用 Colab 要換路徑使用

#from moviepy.editor import VideoFileClip 爛東西，搞死我，放著紀念

format_list = ['avi','mov','wmv','flv','asf', 'mkv']

file_name=os.listdir('input_folder')
video_name=[]
for k in file_name:
    dd=k.split(".")
    if dd[-1]=="avi":
        video_name.append(k)

#print(video_name)

out=[]
for k in video_name:
    k=k.split(".")
    out.append(k[0]+"_new")
print(out,video_name)
i=0
for k in video_name:
    os.popen("ffmpeg -i {input}  {output}.mp4".format(input = k, output = out[i]))
    i+=1
print('慘')

"""
ffmpeg -i {input} -strict -2 {output}.mp4 
如果要直接在terminal上跑，使用上面這行
"""

In [ ]:
####返回原本的位置
os.chdir("..")

In [1]:
####影像修正
####如不需進行鏡頭參數校正 可直截跳到下面

In [3]:
# 檔案路徑與輸出資料夾
import os
video_path = 'your_video.avi'  # 替換為你的 .avi 檔案路徑
output_folder = 'frames'       # 儲存幀的資料夾

In [4]:
#隨機選取照片
import cv2
import random
# 打開影片檔案
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("無法打開影片檔案")
    exit()

# 取得影片資訊
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 隨機選擇 20 個幀號
random_frames = random.sample(range(total_frames), 100)
random_frames.sort()

# 擷取並保存幀
for i, frame_no in enumerate(random_frames):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)  # 移動到指定幀
    ret, frame = cap.read()                    # 讀取幀
    if ret:
        output_path = os.path.join(output_folder, f'frame_{i + 1}.jpg')
        cv2.imwrite(output_path, frame)        # 保存幀為圖片
        print(f"保存幀 {frame_no} 到 {output_path}")
    else:
        print(f"無法讀取幀 {frame_no}")

# 釋放資源
cap.release()
print("完成！")

ModuleNotFoundError: No module named 'cv2'

In [8]:
#利用照片進行校正


# 設定棋盤格的內部角點數量 (cols, rows)
chessboard_size = (9, 6)

#進入資料夾
os.chdir(output_folder)

import numpy as np
import glob
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('*.jpg')
 
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
 
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
 
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

FileNotFoundError: [Errno 2] No such file or directory: 'frames'

In [7]:
#儲存參數
print("相機矩陣 (Camera Matrix):\n", mtx)
print("畸變係數 (Distortion Coefficients):\n", dist)

with open('data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.write([mtx,dist])

NameError: name 'mtx' is not defined

In [10]:
#校正影片
import numpy as np

#讀取之前的參數
with open('data.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    data = list(reader)
    mtx=data[0]
    dist=data[1]
    
# 確保輸出資料夾存在
os.makedirs(output_folder, exist_ok=True)


# 取得所有影片檔案
video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4'))]

# 遍歷所有影片
for video_file in video_files:
    input_path = os.path.join(input_folder, video_file)
    output_path = os.path.join(output_folder, f"undistorted_{video_file}")

    print(f"處理影片: {video_file} -> {output_path}")

    # 讀取影片
    cap = cv2.VideoCapture(input_path)

    # 取得影片的基本資訊
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 定義影片輸出編碼格式
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 格式
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # 計算新的相機矩陣
    new_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, 
                                                           (frame_width, frame_height), 1, 
                                                           (frame_width, frame_height))

    # 預先計算映射表
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, 
                                              new_mtx, (frame_width, frame_height), cv2.CV_32FC1)

    # 讀取並處理影片
    while cap.isOpened():#確定打開影片
        ret, frame = cap.read()
        if not ret:
            break  # 影片播放完畢

        # 校正畫面
        undistorted_frame = cv2.remap(frame, mapx, mapy, cv2.INTER_LINEAR)

        # 寫入校正後的影片
        out.write(undistorted_frame)

    # 釋放資源
    cap.release()
    out.release()
    cv.destroyAllWindows()
    
    print(f"✅ 已完成: {output_path}")

print(" 所有影片處理完成！")

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'